# Numpy Array Operations:  Axes, Broadcasting, Matrix Type

In this note we cover a few concepts about `numpy` multi-dimensional arrays in more detail:
* Using the `axis` feature 
* Python broadcasting

We will need both of these for performing many of the numerical operations for the ML class.

As usual, we begin by loading the `numpy` package.

In [ ]:
import numpy as np

## Axis Parameter

Many operations in the `numpy` package can take an optional `axis` parameter to specify which dimensions the operation is to be applied.  This is extremely useful for multi-dimensional data.  To illustrate the `axis` parameter, consider a matrix the `(3,2)` array `X` defined as:

In [ ]:
X = np.array([[0,1],[2,3],[4,5]])
print(X)

An operation like `np.mean` or `np.sum` takes the mean or sum of *all* elements in the array -- from all rows and columns. 

In [ ]:
print(np.mean(X))
print(np.sum(X))

To take only the `sum` along each column, we can use the `axis` parameter.

In [ ]:
print(np.sum(X,axis=0))

Since `X` has shape `(3,2)`, the output `np.sum(X,axis=0)` is of shape `(2,)`.  Similarly, we can take the `sum` along each row:

In [ ]:
print(np.sum(X,axis=1))

You can apply this to higher-order arrays. Here we create a 3D array to see.

In [ ]:
X = np.arange(24).reshape(2,3,4)  # shape = (2,3,4)
print(X)

In [ ]:
Y1 = np.sum(X,axis=0)             # shape = (3,4)
Y2 = np.sum(X,axis=1)             # shape = (2,4)
print('Y1 = ')
print(Y1)
print('Y2 = ')
print(Y2)

## Broadcasting

**Broadcasting** is a useful tool in Python for performing operations on matrices. It generalizes the useful fact that, if we multiply a numpy array by a scalar, Python knows that we want to multiply *every entry* by that scalar.  

In [ ]:
a = np.array([[1,2,3],[4,5,6]])
b = 2
print(b*a)
print(a*b)

### Example 1:  Mean Removal

Suppose that `X` is a data matrix of shape `(n,p)`.  That is, there are `n` data points and `p` features per point.  Often, we have to remove the mean from each feature.  That is, we want to compute the mean for each feature and then remove the mean from each column.  We could do this with a for-loop as:

In [ ]:
# Generate some random data
n = 100
p = 5
X = np.random.rand(n,p)

In [ ]:
Xm = np.zeros(p)      # Mean for each feature
X_demean = np.zeros((n,p))  # Transformed features with the means removed
for j in range(p):
    Xm[j] = np.mean(X[:,j])
    for i in range(n):
        X_demean[i,j] = X[i,j] - Xm[j]
print(X_demean[0:7,:]) # print the first several rows of the result to compare to later

The code below does this without a for loop using the `axis` parameter and broadcasting.

In [ ]:
# Compute the mean per column using the axis command
Xm = np.mean(X,axis=0)  # This is a p-dim matrix
print(Xm)

To use broadcasting we will need to convert Xm to a 2 dimensional ndarray. We can do this with the `Xm[None,:]` operation, which returns a `(1,p)` shape array.

In [ ]:
print(Xm[None,:])

Using Python broadcasting, we can then subtract the `Xm[None,:]` from `X`. These array are different sizes -- `Xm[None,:]` has one row while `X` has n. But Python automatically figures out that, since the number of columns match, we want to substract `Xm[None,:]` off of every row in X.

In [ ]:
# Subtract the mean
X_demean = X - Xm[None,:]
print(X_demean[0:7,:])

### Example 2:  Standardizing variables

A variant of the above example is to *standardize* the features, where we compute the transform variables,

    Z[i,j] = (X[i,j] - Xm[j])/ Xstd[j]
    
where `Xstd[j]` is the standard deviation per feature.  This can be done as follows:

In [ ]:
Xstd = np.std(X,axis=0)
Z = (X-Xm[None,:])/Xstd[None,:]

### Example 3:  Outer product

The *outer product* of vectors `x` and `y` is the matrix `Z[i,j] = x[i]y[j]`.  This can be performed in one line as follows

In [ ]:
# Some random data
nx = 100
ny = 10
x = np.random.rand(nx)
y = np.random.rand(ny)

# Compute the outer product in one line
Z = x[:,None]*y[None,:]


Here:

     x[:,None] # Has shape (nx,  1)
     y[None,:] # Has shape ( 1, ny)
     
So, with python broadcasting:

     Z = x[:,None]*y[None,:] # has shape (nx,  ny)


**Exercise 1:**  Given a matrix `X`, compute the matrix `Y`, where the rows of `X` are normaized to have norm one.  That is:

     Y[i,j] = X[i,j] / sum_j X[i,j]   

In [ ]:
X = np.random.rand(4,3)
# Y = ...

**Exercise 2:** Diagonal multiplication.  Given a matrix `X` and a vector `d`, compute `Y = diag(d)*X`.

In [ ]:
X = np.random.rand(5,3)
d = np.random.rand(5)
# Y = ...

## Matrix operations with numpy 
Python broadcasting is great, but sometimes it does exactly the wrong thing! If you have a column vector `z` and a matrix `X`, `X*z` won't compute the matrix vector product, but rather use broadcasting to scales `X`'s rows. Similarly for matrix `X` and `Y`, `X*Y` does not compute an actual matrix product.

In [ ]:
X = np.array([[1,2],[3,4]])
z = np.array([[2],[3]])
Y = np.array([[-1,-1],[-1,-1]])
print(str(X)+'\n\n'+str(z)+'\n\n'+str(Y))

In [ ]:
print(str(X*z));print()
print(X*Y)

To compute actually matrix/matrix and matrix/vector products, you can use the `dot` operation.

In [ ]:
print(np.dot(X,z));print()
print(np.dot(np.transpose(z),z));print()
print(np.dot(X,Y));print()

Alternatively, a much cleaner approach is to use numpy's `@` operator. This operator works directly on 2D ndarrays with compatible dimensions. **You do not need to convert to a matrix type as we did in the other demo**. Apparently that approach is outdated, and the `@` operator is now preferred. 

In [ ]:
print(X@z);print()
print(np.transpose(z)@z));print()
print(X@Y)

Note that `@` between a row and column vector gives an alternative way to compute outerproducts.

In [ ]:
print(z@np.transpose(z))